In [1]:
# -*- coding: utf-8 -*-
"""
created: Sat Mar 21 10:55:27 2020

author: Annette Stellema (astellemas@gmail.com)


"""

import numpy as np
import xarray as xr
from pathlib import Path
from itertools import groupby
from datetime import timedelta
import matplotlib.pyplot as plt
from main import paths, lx
from itertools import groupby


def find_runs(x):
    """Find runs of consecutive items in an array."""

    # ensure array
    x = np.asanyarray(x)
    if x.ndim != 1:
        raise ValueError('only 1D array supported')
    n = x.shape[0]

    # handle empty array
    if n == 0:
        return np.array([]), np.array([]), np.array([])

    else:
        # find run starts
        loc_run_start = np.empty(n, dtype=bool)
        loc_run_start[0] = True
        np.not_equal(x[:-1], x[1:], out=loc_run_start[1:])
        run_starts = np.nonzero(loc_run_start)[0]

        # find run values
        run_values = x[loc_run_start]

        # find run lengths
        run_lengths = np.diff(np.append(run_starts, n))

        return run_values, run_starts, run_lengths


def nino_events(oni):

    dn = xr.full_like(oni, 0)
    dn[oni >= 0.5] = 1
    dn[oni <= -0.5] = 2
    run_values, run_starts, run_lengths = find_runs(dn)
    nino = []
    nina = []
    for i, l in enumerate(run_lengths):
        if l >= 5 and run_values[i] == 1:
            j = run_starts[i]
            nino.append([dn.Time[j].dt.strftime('%Y-%m-%d').item(),
                         dn.Time[j+l-1].dt.strftime('%Y-%m-%d').item()])
        elif l >= 5 and run_values[i] == 2:
            j = run_starts[i]
            nina.append([dn.Time[j].dt.strftime('%Y-%m-%d').item(),
                         dn.Time[j+l-1].dt.strftime('%Y-%m-%d').item()])

    return nino, nina

def plot_oni_valid(ds, da, add_obs_ev=False):
    nino1, nina1 = nino_events(ds.oni)



    fig, ax = plt.subplots(figsize=(10, 5))
    ax.set_title('Observed and modelled ENSO events', loc='left')

    plt.plot(da.Time, da.oni, color='red', label='NOAA OISST')
    plt.plot(ds.Time, ds.oni, color='k', label='OFAM')

    for y in [0.5, -0.5]:
        plt.hlines(y=y, xmax=ds.Time[-1], xmin=ds.Time[0],
                   linewidth=1, color='blue', linestyle='--')


    for nin, color in zip([nino1, nina1], ['red', 'blue']):
        for x in range(len(nin)):
            ax.axvspan(np.datetime64(nin[x][0]), np.datetime64(nin[x][1]),
                        alpha=0.15, color=color)
    if add_obs_ev:
        nino2, nina2 = nino_events(da.oni)
        for nin, color in zip([nino2, nina2], ['darkred', 'darkblue']):
            for x in range(len(nin)):
                ax.axvspan(np.datetime64(nin[x][0]), np.datetime64(nin[x][1]),
                            alpha=0.1, color=color, hatch='/')

    ax.set_xlim(xmax=ds.Time[-1], xmin=ds.Time[0])
    plt.ylabel('Oceanic Niño Index [°C]')
    plt.legend(fontsize=10, loc=1)
    plt.savefig(fpath/'valid/oni_ofam_noaa.png')

    return


# Path to save figures, save data and OFAM model output.
fpath, dpath, xpath, lpath, tpath = paths()

ds = xr.open_dataset(dpath/'ofam_sst_anom_nino34_hist.nc')
da = xr.open_dataset(dpath/'noaa_sst_anom_nino34.nc')

ds = ds.sel(Time=slice('1981-11-01', '2012-12-01'))
da = da.sel(time=slice('1981-11-01', '2012-12-01')).rename({'time': 'Time'})




In [53]:
files = []
x = 0
exp = lx['years'][x]
for y in range(exp[0], 1991 + 1):
    for m in range(1, 13):
        files.append(str(xpath/('ocean_{}_{}_{:02d}.nc'.format('u', y, m))))
du = xr.open_mfdataset(files, combine='by_coords')

In [ ]:
from main import timer
import time
# plot_oni_valid(ds, da)
lon = 220

# du = du.sel(st_ocean=slice(2.5, 373), xu_ocean=lx['lons']).sel(yu_ocean=0,
#             method='nearest').u
# du = du.resample(Time='MS').mean()

In [74]:
nino, nina = nino_events(da.oni)
enso = xr.DataArray(np.empty((2, *du[0].shape)).fill(np.nan), 
                 coords=[('nin', ['nino', 'nina']), ('st_ocean', du.st_ocean.values), 
                         ('xu_ocean', du.xu_ocean.values)])
t1 = time.time()
for ix, x in enumerate(lx['lons']):
    tx = time.time()
    print(x)
    for nin, dv in zip([nino, nina], [elv, lav]):
        print('nin')
        tn = time.time()
        for iz, z in enumerate(du.st_ocean):
            tmp = []
            tz = time.time()
            for i in range(len(nin)):
                tmp.append(du.sel(Time=slice(nin[i][0], nin[i][1]),
                                    st_ocean=z, xu_ocean=x).values)
            timer(tz, 'z')
            dv[iz, ix] = np.nanmean(tmp)
        timer(tn, 'nin')
    timer(tx, x)
timer(t1)

165
nin


KeyboardInterrupt: 

In [62]:
elv[:,0].values

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan], dtype=float32)

In [55]:
len(du.sel(Time=slice(nin[ix][0], nin[ix][1])).Time)

13

In [64]:
np.nanmean(tmp)

-0.10819699

In [1]:
import sys
import gsw
import numpy as np
import xarray as xr
import matplotlib.colors
from argparse import ArgumentParser
import matplotlib.pyplot as plt
from main import paths, im_ext, idx_1d, lx, width, height, LAT_DEG, SV
from main_valid import EUC_bnds_static, EUC_bnds_grenier, EUC_bnds_izumo


In [8]:
path, dpath, xpath, lpath, tpath = paths()

fileu, files, filet = [], [], []
method='izumo'
exp = 0
for y in range(lx['years'][exp][0], lx['years'][exp][0]+1):
    for m in range(1, 5):
        fileu.append(str(xpath/'ocean_u_{}_{:02d}.nc'.format(y, m)))
        if method != 'static':
            files.append(str(xpath/('ocean_salt_{}_{:02d}.nc'.format(y, m))))
            filet.append(str(xpath/('ocean_temp_{}_{:02d}.nc'.format(y, m))))


du = xr.open_mfdataset(fileu, combine='by_coords')
if method != 'static':
    ds = xr.open_mfdataset(files, combine='by_coords')
    dt = xr.open_mfdataset(filet, combine='by_coords')


dy = LAT_DEG*0.1
dz = [(du.st_ocean[z+1] - du.st_ocean[z]).item()
      for z in range(0, len(du.st_ocean)-1)]

if method == 'static':
    z1, z2, lat = 25, 350, 2.6
    print('{}: 165 started'.format(method))
    dx_165 = EUC_bnds_static(du, lon=165, z1=z1, z2=z2, lat=lat)
    print('{}: 190 started'.format(method))
    dx_190 = EUC_bnds_static(du, lon=190, z1=z1, z2=z2, lat=lat)
    print('{}: 220 started'.format(method))
    dx_220 = EUC_bnds_static(du, lon=220, z1=z1, z2=z2, lat=lat)

elif method == 'izumo':
#     dx_165 = EUC_bnds_izumo(du, dt, ds, lon=165)
#     dx_190 = EUC_bnds_izumo(du, dt, ds, lon=190)
    dx_220 = EUC_bnds_izumo(du, dt, ds, lon=220)

elif method == 'grenier':
#     print('{}: 165 started'.format(method))
#     dx_165 = EUC_bnds_grenier(du, dt, ds, lon=165)
#     print('{}: 190 started'.format(method))
#     dx_190 = EUC_bnds_grenier(du, dt, ds, lon=190)
    print('{}: 220 started'.format(method))
    dx_220 = EUC_bnds_grenier(du, dt, ds, lon=220)


dtx = xr.Dataset()
dtx['uvo'] = xr.DataArray(np.zeros((len(dx_220.Time), 3)),
                          coords=[('Time', dx_220.Time),
                                  ('xu_ocean', lx['lons'])])

dz_i, dz_f = [],  []
for i, lon, dx in zip(range(1), [220], [dx_220]):
    dz_i.append(idx_1d(du.st_ocean, dx.st_ocean[0]))
    dz_f.append(idx_1d(du.st_ocean, dx.st_ocean[-1]))

    dr = (dx*dy).sum(dim='yu_ocean')
    if method != 'grenier':
        dtx.uvo[:, i] = (dr[:, :]*dz[dz_i[i]:dz_f[i]+1]).sum(dim='st_ocean')
    else:
        dtx.uvo[:, i] = (dr[:, :-1]*dz[dz_i[i]:dz_f[i]]).sum(dim='st_ocean')

dtx['uvo'].attrs['long_name'] = ('OFAM3 EUC daily transport {} boundaries'
                                 .format(method))
dtx['uvo'].attrs['units'] = 'm3/sec'

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-20.01/lib/python3.7/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in greater
  return func(*(_execute_task(a, cache) for a in args))


In [7]:
dtx.uvo

<xarray.DataArray 'uvo' (Time: 181, xu_ocean: 3)>
array([[22828183.05105576,        0.        ,        0.        ],
       [22415019.12422353,        0.        ,        0.        ],
       [22109332.7437408 ,        0.        ,        0.        ],
       [22303567.29296303,        0.        ,        0.        ],
       [23359429.24409419,        0.        ,        0.        ],
       [24113742.09049642,        0.        ,        0.        ],
       [24269126.93043362,        0.        ,        0.        ],
       [26069597.49888872,        0.        ,        0.        ],
       [27070694.56607351,        0.        ,        0.        ],
       [28377543.54755393,        0.        ,        0.        ],
       [29748049.04100668,        0.        ,        0.        ],
       [29985734.64046276,        0.        ,        0.        ],
       [28663015.85784817,        0.        ,        0.        ],
       [27712971.39595675,        0.        ,        0.        ],
       [25121762.26986033,        0.        ,        0.        ],
       [23361602.58950305,        0.        ,        0.        ],
       [23888205.25074208,        0.        ,        0.        ],
       [24712742.90557137,        0.        ,        0.        ],
       [25230157.47696227,        0.        ,        0.        ],
       [24573862.89215056,        0.        ,        0.        ],
       [23287668.16935325,        0.        ,        0.        ],
       [22523883.49926978,        0.        ,        0.        ],
       [22570361.64524255,        0.        ,        0.        ],
       [24075299.65558673,        0.        ,        0.        ],
       [24792706.17665184,        0.        ,        0.        ],
       [23328722.54354389,        0.        ,        0.        ],
       [20027512.71349661,        0.        ,        0.        ],
       [20222012.57956837,        0.        ,        0.        ],
       [21768462.65530518,        0.        ,        0.        ],
       [23934125.31948805,        0.        ,        0.        ],
       [26070523.97504905,        0.        ,        0.        ],
       [25583783.82256855,        0.        ,        0.        ],
       [24417930.90082206,        0.        ,        0.        ],
       [25539671.3258419 ,        0.        ,        0.        ],
       [26101628.64251918,        0.        ,        0.        ],
       [24750778.11016184,        0.        ,        0.        ],
       [22914818.92992181,        0.        ,        0.        ],
       [22693754.39248335,        0.        ,        0.        ],
       [24067634.16353284,        0.        ,        0.        ],
       [27387502.90945345,        0.        ,        0.        ],
       [28759040.57394267,        0.        ,        0.        ],
       [30975417.12202985,        0.        ,        0.        ],
       [28357067.8031868 ,        0.        ,        0.        ],
       [27900191.5169244 ,        0.        ,        0.        ],
       [29634747.35788655,        0.        ,        0.        ],
       [30811669.34685704,        0.        ,        0.        ],
       [30509407.77763756,        0.        ,        0.        ],
       [30564523.83577991,        0.        ,        0.        ],
       [32783602.20893285,        0.        ,        0.        ],
       [35849437.53596874,        0.        ,        0.        ],
       [40674601.7308968 ,        0.        ,        0.        ],
       [43134332.93372315,        0.        ,        0.        ],
       [43059233.74746693,        0.        ,        0.        ],
       [42444348.31175915,        0.        ,        0.        ],
       [44662799.51162641,        0.        ,        0.        ],
       [47172672.86655608,        0.        ,        0.        ],
       [48568848.85428938,        0.        ,        0.        ],
       [47988183.68316734,        0.        ,        0.        ],
       [48283659.53119126,        0.        ,        0.        ],
       [49392737.86241731,        0.        ,        0. 

In [9]:
dtx.uvo/1e6

<xarray.DataArray 'uvo' (Time: 120, xu_ocean: 3)>
array([[23.6093513 ,  0.        ,  0.        ],
       [23.06169524,  0.        ,  0.        ],
       [21.36652259,  0.        ,  0.        ],
       [20.77216981,  0.        ,  0.        ],
       [19.50155   ,  0.        ,  0.        ],
       [17.91010825,  0.        ,  0.        ],
       [17.01995514,  0.        ,  0.        ],
       [17.92715962,  0.        ,  0.        ],
       [18.70617756,  0.        ,  0.        ],
       [19.32876663,  0.        ,  0.        ],
       [19.84657703,  0.        ,  0.        ],
       [20.24398174,  0.        ,  0.        ],
       [20.49243435,  0.        ,  0.        ],
       [20.89510684,  0.        ,  0.        ],
       [21.19215953,  0.        ,  0.        ],
       [20.26497325,  0.        ,  0.        ],
       [20.20918347,  0.        ,  0.        ],
       [22.63574918,  0.        ,  0.        ],
       [24.65876816,  0.        ,  0.        ],
       [25.74027388,  0.        ,  0.        ],
       [24.86060291,  0.        ,  0.        ],
       [23.23118891,  0.        ,  0.        ],
       [22.02287084,  0.        ,  0.        ],
       [22.44399786,  0.        ,  0.        ],
       [23.68259938,  0.        ,  0.        ],
       [23.50528926,  0.        ,  0.        ],
       [22.95053949,  0.        ,  0.        ],
       [24.1900785 ,  0.        ,  0.        ],
       [26.47738932,  0.        ,  0.        ],
       [28.0611978 ,  0.        ,  0.        ],
       [26.88331821,  0.        ,  0.        ],
       [23.31875532,  0.        ,  0.        ],
       [19.60581396,  0.        ,  0.        ],
       [20.01595582,  0.        ,  0.        ],
       [21.68436237,  0.        ,  0.        ],
       [22.14074394,  0.        ,  0.        ],
       [20.99170308,  0.        ,  0.        ],
       [19.11289092,  0.        ,  0.        ],
       [19.62712663,  0.        ,  0.        ],
       [19.17836686,  0.        ,  0.        ],
       [19.13632326,  0.        ,  0.        ],
       [20.01211848,  0.        ,  0.        ],
       [20.74659604,  0.        ,  0.        ],
       [22.10113742,  0.        ,  0.        ],
       [25.52382739,  0.        ,  0.        ],
       [26.89454896,  0.        ,  0.        ],
       [27.0417177 ,  0.        ,  0.        ],
       [26.60375673,  0.        ,  0.        ],
       [26.41938497,  0.        ,  0.        ],
       [28.96158254,  0.        ,  0.        ],
       [32.07834688,  0.        ,  0.        ],
       [36.41095104,  0.        ,  0.        ],
       [38.52204031,  0.        ,  0.        ],
       [38.97748406,  0.        ,  0.        ],
       [40.24355284,  0.        ,  0.        ],
       [43.25462332,  0.        ,  0.        ],
       [45.96322207,  0.        ,  0.        ],
       [48.7845269 ,  0.        ,  0.        ],
       [51.75772632,  0.        ,  0.        ],
       [51.84405746,  0.        ,  0.        ],
       [51.96872977,  0.        ,  0.        ],
       [51.43489497,  0.        ,  0.        ],
       [50.94718592,  0.        ,  0.        ],
       [49.85254597,  0.        ,  0.        ],
       [50.59202971,  0.        ,  0.        ],
       [50.34412215,  0.        ,  0.        ],
       [50.21829144,  0.        ,  0.        ],
       [48.32943527,  0.        ,  0.        ],
       [44.4943484 ,  0.        ,  0.        ],
       [43.28248361,  0.        ,  0.        ],
       [42.67235907,  0.        ,  0.        ],
       [42.96500489,  0.        ,  0.        ],
       [44.88279618,  0.        ,  0.        ],
       [46.64090957,  0.        ,  0.        ],
       [45.88606324,  0.        ,  0.        ],
       [45.54421719,  0.        ,  0.        ],
       [46.54638098,  0.        ,  0.        ],
       [47.67869956,  0.        ,  0.        ],
       [47.16439231,  0.        ,  0.        ],
       [46.20185413,  0.        ,  0.        ],
       [44.66828384,  0.        ,  0.        ],
       [43.80358094,  0.        ,  0.        ],
       [43.601

In [93]:
enso = xr.DataArray(np.empty((2, *du[0].shape)).fill(np.nan), 
                 coords=[('nin', ['nino', 'nina']), ('st_ocean', du.st_ocean.values), 
                         ('xu_ocean', du.xu_ocean.values)])

In [90]:
du[0].shape

(30, 3)

In [11]:
ds = xr.open_dataset('/g/data1/rr7/ERA_INT/ERA_INT/ERA_INT_uas_1981.nc')
ds.uas

<xarray.DataArray 'uas' (time: 12, lat: 121, lon: 240)>
[348480 values with dtype=float32]
Coordinates:
  * lon      (lon) float32 0.0 1.5 3.0 4.5 6.0 ... 352.5 354.0 355.5 357.0 358.5
  * lat      (lat) float32 -90.0 -88.5 -87.0 -85.5 -84.0 ... 85.5 87.0 88.5 90.0
  * time     (time) datetime64[ns] 1981-01-01 1981-02-01 ... 1981-12-01
Attributes:
    standard_name:  eastward_wind
    long_name:      Eastward Near-Surface Wind Speed
    units:          m s-1
    comment:        sampled synoptically.
    name:           uas